In [1]:
# Short script to test the projected gradient method against other
# algorithms.
using Distributions
using LowRankApprox
include("../code/datasim.jl");
include("../code/likelihood.jl");
include("../code/mixEM.jl");
include("../code/mixGD.jl");
include("../code/mixSQP.jl");

# Initialize the sequence of pseudorandom numbers.
srand(1);

# Generate a data set with n = 50,000.
z = normtmixdatasim(round(Int,1000));

# Compute the 50,000 x 20 likelihood matrix.
sd = autoselectmixsd(z,nv = 10000);
L  = normlikmatrix(z,sd = sd);

In [28]:
# Initialize the sequence of pseudorandom numbers.
srand(1);

# Generate a data set with n = 50,000.
z = normtmixdatasim(round(Int,2000));

# Compute the 50,000 x 20 likelihood matrix.
sd = autoselectmixsd(z,nv = 1000);
L  = normlikmatrix(z,sd = sd);

In [26]:
# Initialize the sequence of pseudorandom numbers.
srand(1);

# Generate a data set with n = 50,000.
z = normtmixdatasim(round(Int,50000));

# Compute the 50,000 x 20 likelihood matrix.
sd = autoselectmixsd(z,nv = 100);
L  = normlikmatrix(z,sd = sd);

In [29]:
# Run the mix-SQP algorithm.
@time outsqp = mixSQP(L,lowrank = "none", eps = 1e-8, maxqpiter = 300, verbose = false);
@time outsqp2 = mixSQP(L,lowrank = "qr", eps = 1e-8, maxqpiter = 300, verbose = false);

# Run the EM algorithm.
@time xem, fem = mixEM(L,maxiter = 1000,tol = 1e-4);

# Run the projected gradient descent method.
@time xpgd, fpgd = mixGD(L,maxiter = 2000, alpha = 1);

# Compare the quality of the solutions.
@printf "Objective at SQP  solution: %0.12e\n" mixobjective(L,outsqp["x"])
@printf "Objective at SQP2 solution: %0.12e\n" mixobjective(L,outsqp2["x"])
@printf "Objective at EM   solution:  %0.12e\n" mixobjective(L,xem)
@printf "Objective at PGD  solution: %0.12e\n" mixobjective(L,xpgd)

  3.189305 seconds (80.28 k allocations: 1.997 GiB, 59.48% gc time)
  1.828580 seconds (263.22 k allocations: 1005.268 MiB, 9.66% gc time)
  0.216901 seconds (10.26 k allocations: 230.245 MiB, 12.04% gc time)
  5.916267 seconds (575.72 k allocations: 498.929 MiB, 0.95% gc time)
Objective at SQP  solution: 4.083035719882e+02
Objective at SQP2 solution: 4.083035720771e+02
Objective at EM   solution:  4.310728523556e+02
Objective at PGD  solution: 4.090889249050e+02
